In [15]:
import setup
setup.init_django()

In [16]:
from market.models import StockQuote

In [17]:
from django.db.models import Avg, F, RowRange, Window
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [18]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=30)
end_date = now

qs = StockQuote.objects.filter(time_range=(start_date, end_date))

In [19]:
total = 0
for obj in qs:
    total += obj.close_price

In [20]:
total /qs.count()

Decimal('171.5885245675072581999664083')

In [21]:
qs.values('company').annotate(avg_price=Avg('close_price'))

<QuerySet [{'company': 2, 'avg_price': Decimal('98.4583735781906185')}, {'company': 1, 'avg_price': Decimal('208.7756351527435935')}]>